In [1]:
import sys
from pathlib import Path
import functools

import mujoco
from brax import envs
import jax
import jax.numpy as jp

from envs import register_inverted_pendulum

import shac.networks as shac_networks
# import shac.train as  shac
from shac.train_class import SHAC

In [2]:
env_name = 'inverted_pendulum_diffble'

log_sigma = None
make_networks_factory = functools.partial(
    shac_networks.make_shac_networks,
        policy_hidden_layer_sizes=(64, 64),
        value_hidden_layer_sizes =(64, 64),
        scalar_var=False,
        layer_norm=True)


In [3]:
# Training Parameters
unroll_length = 32 # Somewhat learns at 1. Doesn't at 8. Blows up at 16.

num_epochs = int(500 * (32 / unroll_length))
num_evals = num_epochs + 1
num_envs = 128

num_timesteps = (num_envs*unroll_length)*num_epochs

num_critic_minibatches = 4
num_critic_data = num_envs*unroll_length
critic_batch_size = num_critic_data / num_critic_minibatches


In [4]:

env = envs.get_environment(env_name)
eval_env = envs.get_environment(env_name)

In [5]:
kwargs = dict( 
  num_timesteps=num_timesteps, log_sigma=log_sigma,
  num_evals=num_evals, use_tbx=True, tbx_logdir=f'{env_name}_log',
  tbx_experiment_name="", reward_scaling=1, 
  num_eval_envs = 64, num_envs=num_envs,
  unroll_length=unroll_length, critic_batch_size=critic_batch_size,
  critic_epochs = 16,target_critic_alpha = 0.2,
  discounting = 0.99, lambda_ = 0.95,
  normalize_observations=True,
  episode_length=1000, 
  network_factory = make_networks_factory, 
  seed=0, entropy_cost = 1e-6,
  resample_init=True,
  scramble_initial_times=True,
  num_grad_checks=None,
  save_all_checkpoints=True,
  save_all_policy_gradients=False
  )

trainer = SHAC(environment=env, eval_env=eval_env, **kwargs)

Env steps per training step: 4096
Training steps per epoch: 1
Critic minibatches per critic epoch: 4


In [6]:
make_inference_fn, policy_params, value_params, _= trainer.train()